In [2]:
import numpy as np
from load_data import load
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Conv1D, Add, Reshape
from keras.layers import AveragePooling2D, UpSampling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import keras.backend as K
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2DTranspose
from keras.layers.core import Lambda
from keras.models import Model
import datetime

K.set_image_data_format('channels_last')
print('Done')

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Done


In [3]:
[X_train,Y_train,X_test,Y_test] = load("blast_tab_1hit.out")

Loading data...
58863 sequences were uploaded

Maximum sequence length in is 308
Maximum sequence length out is 308

Converting to one-hot...
Done
Converting to one-hot...
Done


In [4]:
m = X_train.shape[0]
print("There are " + str(m) + " training examples")
print("There are " + str(X_test.shape[0]) + " testing examples")
print("There are " + str(X_train.shape[1]) + " classes: A, C, G, T")
max_length = max(X_train.shape[2],X_test.shape[2])
print("The longest sequence is " + str(max_length) + " nucleotides long")
print(X_train.shape)

There are 55996 training examples
There are 2867 testing examples
There are 4 classes: A, C, G, T
The longest sequence is 308 nucleotides long
(55996, 4, 308, 1)


In [5]:
print('Permuting...')
np.random.seed(0)
rand_perm = np.random.rand(m).argsort()
np.take(X_train,rand_perm,axis=0,out=X_train)
print("finished X")
np.take(Y_train,rand_perm,axis=0,out=Y_train)
print("finished Y")

Permuting...
finished X
finished Y


In [6]:
# Visualize data sets to ensure they appear as anticipated
sample = 300
print(X_train[sample,:,10:30,0])
print(Y_train[sample,:,10:30,0])

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1.]]
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1.]]


In [7]:
ngf = 16
input_nc = 1
output_nc = 1
n_blocks_gen = 9

# started from: https://blog.sicara.com/keras-generative-adversarial-networks-image-deblurring-45e3ab6977b5
def Model_1(input_shape):
    """Build generator architecture."""
    # Current version : ResNet block
    X_input = Input(input_shape)
    
    # X = ZeroPadding2D((0, 3))(X_input) # allows 'valid' in conv0 to keep length and collapse one-hot

    X = Conv2D(128, (4, 7), strides = (1, 1), padding = 'same', name = 'conv0')(X_input)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    
    # X = Conv2D(64, (1, 7), strides = (1, 1), padding = 'same', name = 'conv1')(X)
    # X = BatchNormalization(axis = 3, name = 'bn1')(X)
    # X = Activation('relu')(X)
    
    X = Conv2D(64, (1, 7), strides = (1, 1), padding = 'same', name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    
    X = Dropout(0.3, name = 'dropout')(X)
   
    X = Conv2D(32, (1, 7), strides = (1, 1), padding = 'same', name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(1, (1, 1), strides = (1, 1), padding = 'same', name = 'conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('sigmoid')(X)
    
    model = Model(inputs=X_input, outputs=X, name='Model_1')
    return model

In [8]:
myModel = Model_1((4,max_length,1))
print(myModel.summary())
print('Done!')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4, 308, 1)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 4, 308, 128)       3712      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 4, 308, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 4, 308, 128)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 4, 308, 64)        57408     
_________________________________________________________________
bn1 (BatchNormalization)     (None, 4, 308, 64)        256       
_________________________________________________________________
activation_2 (Activation)    (None, 4, 308, 64)        0         
__________

In [9]:
myModel.compile(optimizer="Adam", loss="mean_squared_error", metrics = ["accuracy"])
print('Done!')

Done!


In [10]:
# Understand baseline accuracy of if model output its own input
diffs = np.absolute(X_test-Y_test);
err = np.sum(diffs)/np.ma.size(X_test);
print("Baseline accuracy if predicting output = input is " + str(1-err))

Baseline accuracy if predicting output = input is 0.9909009938439656


In [240]:
myModel.fit(x = X_train, y = Y_train, epochs = 10, batch_size = 50)

loss_and_acc = myModel.evaluate(X_test, Y_test)
print(loss_and_acc)

currtime = datetime.datetime.now()
fname = "./convWeights/" + currtime.strftime("%m%d-%H%M") + ".hdf5"
print(fname)
myModel.save_weights(fname)

Epoch 1/10
55996/55996 [==============================] - 1629s 29ms/step - loss: 0.0020 - acc: 0.9979
Epoch 2/10
55996/55996 [==============================] - 1623s 29ms/step - loss: 0.0020 - acc: 0.9979
Epoch 3/10
55996/55996 [==============================] - 1626s 29ms/step - loss: 0.0020 - acc: 0.9979
Epoch 4/10
55996/55996 [==============================] - 1633s 29ms/step - loss: 0.0020 - acc: 0.9979
Epoch 5/10
55996/55996 [==============================] - 1619s 29ms/step - loss: 0.0020 - acc: 0.9979
Epoch 6/10
55996/55996 [==============================] - 1620s 29ms/step - loss: 0.0020 - acc: 0.9979
Epoch 7/10
55996/55996 [==============================] - 1620s 29ms/step - loss: 0.0020 - acc: 0.9979
Epoch 8/10
55996/55996 [==============================] - 1620s 29ms/step - loss: 0.0020 - acc: 0.9979
Epoch 9/10
55996/55996 [==============================] - 1900s 34ms/step - loss: 0.0020 - acc: 0.9979
Epoch 10/10
2867/2867 [==============================] - 25s 9ms/step
[0.

In [15]:
preds = myModel.predict(x = X_test)

In [18]:
# Change n to view the predictions, true sequence, and input sequence for a given sample
n = 4
print("Sample number " + str(n+1))
samp_n_pred = preds[n,0:4,:,0]
samp_n_in = X_test[n,0:4,:,0]
samp_n_true = Y_test[n,0:4,:,0]

np.set_printoptions(precision=3, suppress=True)
noise = np.argmax(np.sum(np.square(samp_n_in - samp_n_true),axis=0))
print("The noise was at nucleotide number " + str(noise))
print("The predictions from " + str(noise-2) + " to " + str(noise+2) + " are: ")
print(preds[n,0:4,noise-2:noise+3,0])
print("The true denoised nucleotides from " + str(noise-2) + " to " + str(noise+2) + " are: ")
print(Y_test[n,0:4,noise-2:noise+3,0])
print("The input nucleotides from " + str(noise-2) + " to " + str(noise+2) + " were: ")
print(X_test[n,0:4,noise-2:noise+3,0])

Sample number 5
The noise was at nucleotide number 40
The predictions from 38 to 42 are: 
[[0.999 1.    0.037 0.    0.001]
 [0.    0.001 0.001 0.998 0.004]
 [0.    0.    0.983 0.001 0.988]
 [0.    0.003 0.001 0.001 0.009]]
The true denoised nucleotides from 38 to 42 are: 
[[1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0.]]
The input nucleotides from 38 to 42 were: 
[[1. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]]


In [ ]:
def convert_to_nucs(one_hot):
    rows = np.argmax(one_hot,axis=0)
    conf = np.max(one_hot,axis=0)
    nucs = ''
    for idx in range(one_hot.shape[1]):
        if(conf[idx]>0.3):
            nuc = rows[idx]
        else:
            nuc = '-'
            
        if(nuc == 0):
            nucs = nucs+'A'
        elif(nuc == 1):
            nucs = nucs+'C'
        elif(nuc == 2):
            nucs = nucs+'G'
        elif(nuc == 3):
            nucs = nucs+'T'
        else:
            nucs = nucs+'-'
            
    return nucs

In [28]:
def show_noise(pred,denoised,noisy):
    noise = ''
    for idx in range(len(pred)):
        if(pred[idx] == denoised[idx] and pred[idx] == noisy[idx]):
            noise = noise+'-'
        elif(pred[idx] == denoised[idx]):
            noise = noise+'g'
        else:
            noise = noise+'b'
    return noise        

In [34]:
# Show whole sequence
for n in range(100):
    print("Sample number " + str(n+1))
    samp_n_pred = preds[n,:,:,0]
    samp_n_in = X_test[n,:,:,0]
    samp_n_true = Y_test[n,:,:,0]

    samp_n_pred = convert_to_nucs(samp_n_pred)
    samp_n_true = convert_to_nucs(samp_n_true)
    samp_n_in = convert_to_nucs(samp_n_in)
    noisy = show_noise(samp_n_pred, samp_n_true, samp_n_in)

    print('Predicted:')
    print('Denoised:')
    print('Noisy:')
    print('Noise locations:\n')

    stt = 0
    stp = 100
    while(stp < len(noisy)):
        print(samp_n_pred[stt:stp])
        print(samp_n_true[stt:stp])
        print(samp_n_in[stt:stp])
        print(noisy[stt:stp])
        print('')
        stt = stp
        stp = stt+100



Sample number 1
Predicted:
Denoised:
Noisy:
Noise locations:

ACGGAGGGTCCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTTTGTTAAGCGAGATGTGAAAGCCCCGGGCTCAACCTGGGAATTGC
ACGGAGGGTCCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTTTGTTAAGCGAGATGTGAAAGCCCCGGGCTCAACCTGGGAATTGC
ACGGAGGGTCCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTTTGTTAAGCGAGATGTGAAAGCCCCGGGCTCAACCTGGGAATTGC
----------------------------------------------------------------------------------------------------

ATTTCGAACTGGCGAACTAGAGTCTTGTAGAGGGGGGTAGAATTCCAGGTGTAGCGGTGAAATGCGTAGAGATCTGGAGGAATACCGGTGGCGAAGGCGG
ATTTCGAACTGGCGAACTAGAGTCTTGTAGAGGGGGGTAGAATTCCAGGTGTAGCGGTGAAATGCGTAGAGATCTGGAGGAATACCGGTGGCGAAGGCGG
ATTTCGAACTGGCGAACTAGAGTCTTGTAGAGGGGGGTAGAATTCCAGGTGTAGCGGTGAAATGCGTAGAGATCTGGAGGAATACCGGTGGCGAAGGCGG
----------------------------------------------------------------------------------------------------

CCCCCTGGACAAAGACTGACGCTCAGGCACGAAAGCGTGGGGAGCAAACAG-------------------------------------------------
CCCCCTGGACAAAGACTGACGCTCAGG

Predicted:
Denoised:
Noisy:
Noise locations:

ACGGAGGGTCCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTCTTG--AAGCGAGATGTGAAAGCCCCGGGCTCAACCTGGGAATTG
ACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGGGTGCGCAGGCGGTCTTG-CAAGTCAGATGTGAAAGCCCCGGGCTTAACCTGGGAATTG
ACGGAGGGTCCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGT-TTGTTAAGCGAGATGTGAAAGCCCCGGGCTCAACCTGGGAATTG
---b-----b-------------------------------b-------------g---gb---bb--------------------b-------------

CGTTTGAAACTACAAGGCTAGAGTGCAGCAGAGGGGAGTGGAATTCCATGTGTAGCAGTGAAATGCGTAGAGATGTGGAAGAACACCGATGGCGAAGGCA
CGTTTGAAACTACAAGGCTAGAGTGCAGCAGAGGGGAGTGGAATTCCATGTGTAGCAGTGAAATGCGTAGAGATGTGGAAGAACACCGATGGCGAAGGCA
CGTTTGAAACTACAAGGCTAGAGTGCAGCAGAGGGGAGTGGAATTCCATGTGTAGCAGTGAAATGCGTAGAGATGTGGAAGAACACCGATGGCGAAGGCA
----------------------------------------------------------------------------------------------------

GCTCCCTGGGTTGACACTGACGCTCATGCACGAAAGCGTGGGGAGCAAACAG------------------------------------------------
GCTCCCTGGGTTGACACTGACGCTCATGCACGAAAGCGTGGGG

Predicted:
Denoised:
Noisy:
Noise locations:

ACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCGTCTAGGTGGTTATGTAAGTCTGATGTGAAAATGCAGGGCTCAACTCTGTATTGCG
ACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCGTCTAGGTGGTTATGTAAGTCTGATGTGAAAATGCAGGGCTCAACTCTGTATTGCG
ACGTATGTCACAAGCGTTATCCGGATTTATTGGGCGTAAAGCGCGTCTAGGTGGTTATGTAAGTCTGATGTGAAAATGCAGGGCTCAACTCTGTATTGCG
----------------------------------------------------------------------------------------------------

TTGGAAACTGTGTAACTAGAGTACTGGAGAGGTAAGCGGAACTACAAGTGTAGAGGTGAAATTCGTAGATATTTGTAGGAATGCCGATGGGGAAGCCAGC
TTGGAAACTGTGTAACTAGAGTACTGGAGAGGTAAGCGGAACTACAAGTGTAGAGGTGAAATTCGTAGATATTTGTAGGAATGCCGATGGGGAAGCCAGC
TTGGAAACTGTGTAACTAGAGTACTGGAGAGGTAAGCGGAACTACAAGTGTAGAGGTGAAATTCGTAGATATTTGTAGGAATGCCGATGGGGAAGCCAGC
----------------------------------------------------------------------------------------------------

TTACTAGACAGATACTGACGCTGAGGTGCGAAAGCGTGGGTAGCAAACAG--------------------------------------------------
TTACTAGACAGATACTGACGCTGAAGCGCGAAAGCGTGGGTAG

In [11]:
myModel.load_weights("./convWeights/0520-1212.hdf5")